<a href="https://colab.research.google.com/github/HarshitKmr10/TPS-APRIL/blob/main/TPS_APRIL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle competitions download tabular-playground-series-apr-2022

In [ ]:
!unzip tabular-playground-series-apr-2022

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import numpy as np # linear algebra
import pandas as pd
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")

In [ ]:
 # data processing, CSV file I/O (e.g. pd.read_csv)


import tensorflow as tf
import time, logging, gc
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.metrics import roc_auc_score
np.random.seed(42)
tf.random.set_seed(42)

from tensorflow.keras.layers import *
from tensorflow.keras import *
from tensorflow.keras.callbacks import *
from sklearn.model_selection import KFold, GroupKFold
from tensorflow.keras.metrics import AUC
import matplotlib.pyplot as plt   

from tqdm.notebook import tqdm
import sklearn


from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau

!pip install keras-self-attention
from keras_self_attention import SeqSelfAttention

In [ ]:
train_labels=pd.read_csv("train_labels.csv")

In [ ]:
features  = [col for col in test.columns if col not in ("sequence","step","subject")]

train = pd.merge(train, train_labels,how='left', on="sequence")

# %% [code] {"jupyter":{"outputs_hidden":false}}
 
def addFeatures(df):  
    for feature in features:
        df[feature + '_lag1'] = df.groupby('sequence')[feature].shift(1)
        df.fillna(0, inplace=True)
        df[feature + '_diff1'] = df[feature] - df[feature + '_lag1']    
    return df


train = addFeatures(train)
test = addFeatures(test)

# %% [code] {"jupyter":{"outputs_hidden":false}}
train

# %% [code] {"jupyter":{"outputs_hidden":false}}
train.info()

# %% [code] {"jupyter":{"outputs_hidden":false}}
a=train.groupby(sort=False,by="sequence").mean()

# %% [code] {"jupyter":{"outputs_hidden":false}}
a=train.subject.unique()
a=np.array(a)
len(a)

# %% [code] {"jupyter":{"outputs_hidden":false}}
a.sort()
l=a==[i for i in range(0,672)]
if l.all():
    print("TRue")

# %% [code] {"jupyter":{"outputs_hidden":false}}
train_labels
train[["sequence","subject"]].iloc[60:60+60] 
#implication --> sequence corresponds to an explicit subjects
#train=train.drop("subject",axis=1)
train

# %% [code] {"jupyter":{"outputs_hidden":false}}
train_sequence=[]
train_labels

# %% [code] {"jupyter":{"outputs_hidden":false}}
for i in train.groupby(sort=False,by="sequence"):
    train_sequence.append(i)
train_sequence[1][1].head()

# %% [code] {"jupyter":{"outputs_hidden":false}}
def call(df,seq):
    return df[seq][1]
call(train_sequence,1).head()

# %% [markdown]
# # seperating indexes for 1's and 0's reading to understand the data

# %% [code] {"jupyter":{"outputs_hidden":false}}
train_labels
zeros_index=[]
ones_index=[]
for i in range(len(train_labels)):
    if train_labels["state"].iloc[i]==1:
        ones_index.append(i)
    else:
        zeros_index.append(i)
print(len(ones_index),len(zeros_index))

# %% [markdown]
# almost equal ones and zeros to train :)

# %% [markdown]
# # visualising senosory data

# %% [code] {"jupyter":{"outputs_hidden":false}}
import matplotlib.pyplot as plt
%matplotlib inline

# %% [code] {"jupyter":{"outputs_hidden":false}}
features_sen=[i for i in train.columns if "sensor" in i]
#call(train_sequence,j)["sensor_00"]

# %% [markdown]
# # superimposed reading

# %% [code] {"jupyter":{"outputs_hidden":false}}
def super_imp(arr,tt=train_sequence):
    for i in features_sen:
        plt.figure(figsize=(18, 3))
        for j in arr:#add any to the bracket for comparing in one
            l=call(tt,j)
            l=l.set_index("step")
            plt.plot(l[i])#x=60,y=values of column
        plt.legend(arr)
        plt.title(str(i))
        plt.show()

# %% [markdown]
# # comparsion

# %% [code] {"jupyter":{"outputs_hidden":false}}
from random import shuffle

# %% [markdown]
# # displaying ones

# %% [code] {"jupyter":{"outputs_hidden":false}}
train_df=train.copy()
features=features_sen
shuffle(ones_index)
sequences = ones_index[:4]#[0, 1, 2, 8364, 15404,]
figure, axes = plt.subplots(13, len(sequences), sharex=True, figsize=(16, 16))
for i, sequence in enumerate(sequences):
    for sensor in range(13):
        sensor_name = f"sensor_{sensor:02d}"
        plt.subplot(13, len(sequences), sensor * len(sequences) + i + 1)
        plt.plot(range(60), train_df[train_df.sequence == sequence][sensor_name],
                color=plt.rcParams['axes.prop_cycle'].by_key()['color'][i % 10])
        if sensor == 0: plt.title(f"Sequence {sequence}")
        if sequence == sequences[0]: plt.ylabel(sensor_name)
figure.tight_layout(w_pad=0.1)
plt.suptitle('Selected Time Series', y=1.02)
plt.show()
print(train_labels.loc[sequences,["state"]])
sequences_1=sequences

# %% [markdown]
# # displaying zeros

# %% [code] {"jupyter":{"outputs_hidden":false}}
train_df=train.copy()
features=features_sen
shuffle(zeros_index)
sequences_0 = zeros_index[:8]#[0, 1, 2, 8364, 15404,]
sequences=sequences_0
figure, axes = plt.subplots(13, len(sequences), sharex=True, figsize=(16, 16))
for i, sequence in enumerate(sequences):
    for sensor in range(13):
        sensor_name = f"sensor_{sensor:02d}"
        plt.subplot(13, len(sequences), sensor * len(sequences) + i + 1)
        plt.plot(range(60), train_df[train_df.sequence == sequence][sensor_name],
                color=plt.rcParams['axes.prop_cycle'].by_key()['color'][i % 10])
        if sensor == 0: plt.title(f"Sequence {sequence}")
        if sequence == sequences[0]: plt.ylabel(sensor_name)
figure.tight_layout(w_pad=0.1)
plt.suptitle('Selected Time Series', y=1.02)
plt.show()
print(train_labels.loc[sequences_0,["state"]])

# %% [code] {"jupyter":{"outputs_hidden":false}}
super_imp(sequences_0)

# %% [markdown]
# pass data through a function to remove noise while maintsing integrity of the data

# %% [code] {"jupyter":{"outputs_hidden":false}}
from sklearn.feature_selection import VarianceThreshold
transform = VarianceThreshold(threshold=0.05)
# transform the input data
X_sel = transform.fit_transform(train_df)
X_sel

# %% [code] {"jupyter":{"outputs_hidden":false}}


# %% [code] {"jupyter":{"outputs_hidden":false}}


# %% [code] {"jupyter":{"outputs_hidden":false}}
from statsmodels.tsa.stattools import adfuller
from numpy import log
mydata = pd.read_csv('train.csv', names = ['value'], header = 0) 
mydata
print(1)
#res=adfuller(mydata.value.dropna())
print(2)

#res

# %% [code] {"jupyter":{"outputs_hidden":false}}
#dickey fuller test for stationarity since applying arima model p<0.005
no_adfs=[]
for i in range(25968):
    re=adfuller(mydata["value"][i])
    if re[1]<=0.005:
        pass
    else:
        no_adfs.append(i)
    if i in [25968,int(25968*(0.75)),int(25968*(0.5)),int(25968*(0.25)),0]:
        print(i,end=" ")

# %% [code] {"jupyter":{"outputs_hidden":false}}
len(no_adfs),len(train)
len(mydata.value[0])

# %% [code] {"jupyter":{"outputs_hidden":false}}
train["index"]=range(len(train))
a=train.set_index("sequence",False).drop(no_adfs,axis=0)
b=train.set_index("index")
tt=[]
for i in b.groupby(sort=False,by="sequence"):
    tt.append(i)

# %% [code] {"jupyter":{"outputs_hidden":false}}
train_2=a

# %% [code] {"jupyter":{"outputs_hidden":false}}
#copied code for testing
def BuildNN():
    model = keras.models.Sequential([
        keras.layers.Input(shape=(60, 87)),
        keras.layers.Conv1D(32, 5),
        
        keras.layers.Bidirectional(keras.layers.LSTM(768, return_sequences=True)),
        keras.layers.Bidirectional(keras.layers.LSTM(512, return_sequences=True)),
        keras.layers.Bidirectional(GRU(units=256,return_sequences=True)),
        keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True)),
        keras.layers.MaxPooling1D(),
        keras.layers.Conv1D(64, 3),
        keras.layers.GlobalMaxPooling1D(),
        keras.layers.Dense(150, activation="selu"),
        keras.layers.Dense(100, activation='selu'),
        keras.layers.Dense(1, activation="sigmoid")
    ])

    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=[keras.metrics.AUC()])
    return model

# %% [code] {"jupyter":{"outputs_hidden":false}}
def dnn_model():

    x_input = Input(shape=(60,42))
    
   
    x = Bidirectional(LSTM(512, return_sequences=True), name='BiLSTM1')(x_input)
    x = Bidirectional(LSTM(384, return_sequences=True), name='BiLSTM2')(x)
    #x = SeqSelfAttention(attention_activation='selu',name='attention_weight')(x)
    x = GlobalAveragePooling1D()(x)
    
    x_output = Dense(units=1, activation="sigmoid")(x)
    
    model = Model(inputs=x_input, outputs=x_output, name='alstm_model')
    
    return model

model = dnn_model()

# %% [code] {"jupyter":{"outputs_hidden":false}}
import tensorflow as tf
from tensorflow import keras
from IPython.display import display
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score

# %% [code] {"jupyter":{"outputs_hidden":false}}
'''tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)'''

# %% [markdown]
# # modifying feature 2 in train_2 and test

# %% [code] {"jupyter":{"outputs_hidden":false}}
train_2.head()

# %% [code] {"jupyter":{"outputs_hidden":false}}
"""a=train_2.corr()
import seaborn as sns
plt.figure(figsize=(12,12))
sns.heatmap(a,annot=True)
R_2_n=[]
for i in range(4,len(a["sensor_02"])):
    if (a["sensor_02"][i]**2)**(0.5)>0.001:
        R_2_n.append(i-4)
R_2_n=R_2_n[:-1]
R_2_n"""

# %% [code] {"jupyter":{"outputs_hidden":false}}
"""s_2_n=[list(train_2["sensor_02"])]+[list(train_2["sensor_0"+str(i)]) for i in R_2_n]
trial=train_2.copy()
k=list(np.array(s_2_n[0])*(np.array(s_2_n[1])+(0.2)*np.array(s_2_n[2])))
trial["sensor_02"]=k
#/np.array(s_2_n[3])))
plt.figure(figsize=(12,13))
sns.heatmap(trial.corr(),annot=True)
trial"""

# %% [markdown]
# GOOD CORR IN 2 OBTAINED ,THERFORE NOT GOING TO DROP SENS-2

# %% [code] {"jupyter":{"outputs_hidden":false}}
'''train_2["sensor_02"]=k
s_2_n=[list(test["sensor_02"])]
s_2_n+=[list(test["sensor_0"+str(i)]) for i in R_2_n]
test["sensor_02"]=list(np.array(s_2_n[0])*(np.array(s_2_n[1])+(0.2)*np.array(s_2_n[2])))
train_2.head()
train_2=train_2.drop("index",axis=1)
train_2.head()'''

# %% [markdown]
# not doing any cahnges with sensor 2 as it leads to over fitting

# %% [markdown]
# # Adding other features

# %% [code] {"jupyter":{"outputs_hidden":false}}
#sumation of different columns
s=train_2["sensor_00"]+train_2["sensor_01"]+train_2["sensor_06"]+train_2["sensor_09"]
s1=train_2["sensor_02"]+train_2["sensor_03"]+train_2["sensor_07"]+train_2["sensor_12"]
d=s-s1
train_2["sum_1"]=s
train_2["sum_2"]=s1
train_2["diff"]=d
"""plt.figure(figsize=(16,16))
sns.heatmap(train_2.corr(),annot=True)"""

# %% [markdown]
# good results converting test too

# %% [code] {"jupyter":{"outputs_hidden":false}}
s=test["sensor_00"]+test["sensor_01"]+test["sensor_06"]+test["sensor_09"]
s1=test["sensor_02"]+test["sensor_03"]+test["sensor_07"]+test["sensor_12"]
d=s-s1
test["sum_1"]=s
test["sum_2"]=s1
test["diff"]=d

# %% [markdown]
# # scalling data(standardization)

# %% [code] {"jupyter":{"outputs_hidden":false}}
train_2=train_2.drop("index",axis=1)
train_2=train_2.drop("state",axis=1)

# %% [code] {"jupyter":{"outputs_hidden":false}}
cul=train_2.columns
len(cul)

# %% [code] {"jupyter":{"outputs_hidden":false}}
train_2.columns

# %% [code] {"jupyter":{"outputs_hidden":false}}
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
b=sc.fit_transform(train_2)
a=sc.transform(test)
a=pd.DataFrame(a)
b=pd.DataFrame(b)
test=a.set_axis(cul,axis="columns")
train_2=b.set_axis(cul,axis="columns")

# %% [code] {"jupyter":{"outputs_hidden":false}}
from sklearn.decomposition import PCA
pca = PCA()
train_pca = pca.fit_transform(train_2)

# Convert to dataframe
component_names = [f"PC{i+1}" for i in range(train_pca.shape[1])]
train_pca = pd.DataFrame(train_pca, columns=component_names)
pca = PCA()

test_pca = pca.fit_transform(test)
component_names = [f"PC{i+1}" for i in range(test_pca.shape[1])]
test_pca = pd.DataFrame(test_pca, columns=component_names)

# %% [code] {"jupyter":{"outputs_hidden":false}}


# %% [code] {"jupyter":{"outputs_hidden":false}}
train_2=pd.concat([train_2,train_pca],axis=1)
test=pd.concat([test,test_pca],axis=1)

# %% [code] {"jupyter":{"outputs_hidden":false}}
test.head()

# %% [code] {"jupyter":{"outputs_hidden":false}}
train=train_2
groups = train["sequence"]
train = train.drop(["sequence","subject", "step"], inplace=False, axis=1).values
test = test.drop(["sequence", "subject", "step"], inplace=False, axis=1).values
labels = train_labels["state"]
labels=labels.drop(no_adfs,axis=0)
train = train.reshape(int(len(train)/60), 60, 87)
test = test.reshape(int(len(test)/60), 60, 87)

# %% [code] {"jupyter":{"outputs_hidden":false}}
train.shape
len(train)




In [ ]:
cv_score = 0
test_preds = []
kf = GroupKFold(n_splits=5)
for fold_idx, (train_idx, valid_idx) in enumerate(kf.split(train, labels, groups.unique())):
    
    print("*"*15, f"Fold {fold_idx+1}", "*"*15)
    
    X_train, X_valid = train[train_idx], train[valid_idx]
    y_train, y_valid = labels.iloc[train_idx].values, labels.iloc[valid_idx].values
    
    model = BuildNN()
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics='AUC')
    model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=50, batch_size=256, 
              callbacks=[keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)])
    
    cv_score += roc_auc_score(y_valid, model.predict(X_valid).squeeze())
    
    test_preds.append(model.predict(test).squeeze())
    
print(cv_score/5)

In [ ]:
submission=pd.read_csv("sample_submission.csv")

# %% [code] {"jupyter":{"outputs_hidden":false}}


# %% [code] {"jupyter":{"outputs_hidden":false}}
submission["state"] = sum(test_preds)/5
ans=[]
for i in submission["state"]:
    ans.append(i)

# %% [code] {"jupyter":{"outputs_hidden":false}}
submission.to_csv("submission.csv_36", index=False)
submission